###  Reconocimiento de Patrones y Aprendizaje Automatizado
#### ----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
### Proyecto: Hongos Venenosos y Comestibles
#### ----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
#### Integrantes: 
#### Úrsula Vanessa Sánchez Rodríguez
#### Adame Serrano Eduardo.

#### Inspiración
* ¿Qué tipos de modelos de aprendizaje automático funcionan mejor en este conjunto de datos?

* ¿Qué características son más indicativas de un hongo venenoso?

#### Librerías

Importamos las librerías

In [2]:
import pandas as pd
import numpy as np
from tabulate import tabulate
from sklearn.preprocessing import OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, chi2
from keras.models import Sequential
from keras.layers import Dense

Cargamos el archivo 

In [31]:
%%time
data = pd.read_csv('mushrooms.csv')
data

CPU times: total: 0 ns
Wall time: 17.8 ms


,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8119,e,k,s,n,f,n,a,c,b,y,...,s,o,o,p,o,o,p,b,c,l
8120,e,x,s,n,f,n,a,c,b,y,...,s,o,o,p,n,o,p,b,v,l
8121,e,f,s,n,f,n,a,c,b,n,...,s,o,o,p,o,o,p,b,c,l
8122,p,k,y,n,f,y,f,c,n,b,...,k,w,w,p,w,o,e,w,v,l


#### Análisis exploratorio.

* Información de atributos: (clases = class: comestible = e, venenoso = p)

* Forma de la tapa: campana = b, cónico = c, convexo = x, plano = f, perilla = k, hundido = s

* Cap-surface: fibroso = F, ranuras = g, escamosas = y, lisas = s

* Cap-color: marrón = N, Buff = B, Canela = C, Gris = G, Verde = R, Rosa = P, Púrpura = U, Rojo = E, Blanco = W, Amarillo = Y

* Moretones: moretones = t, no = f

* Olor: almendra = a, anís = L, creosota = C, pescado = y, foul = f, mohoso = m, ninguno = N, picante = P, picante = S

* Gill-attachment = Fijación branquial: attached=a,descending=d,free=f,notched=n

* Gill-spacing= Espaciado branquial: close = c, crowded = w, distante = d

* Gill-size = tamaño de branquias: ancho = b, estrecho = n

* Gill-color = color de branquias: negro = k, marrón = N, buff = B, chocolate = H, gris = G, verde = R, naranja = O, rosa = p, púrpura = U, rojo = E, blanco = W, amarillo = Y

* Forma del tallo: agrandando = e, tapering = t

* Raíz-tallo: bulboso = b, club = c, copa = u, igual = e, rizomorfos = z, rooted = r, falta =?

* Superficie del tallo-por encima del anillo: fibroso = F, escamoso = y, sedoso = K, liso = S

* Superficie del tallo-debajo del anillo: fibroso = F, escamoso = y, sedoso = K, liso = S

* Tallo-color-encima-del-anillo: marrón=N,Buff=B,Canela=C,Gray=G,Orange=O,Pink=P,Red=E,White=W,Yellow=Y

* Tallo-color-debajo-anillo: marrón=N,Buff=B,Canela=C,Gray=G,Orange=O,Pink=P,Red=E,White=W,Yellow=Y

* Tipo de velo: parcial = p, universal = u

* Color del velo: marrón = n, naranja = o, blanco = w, amarillo = y

* Número de anillo: none=n,one=o,two=t

* Tipo de anillo: telaraña = c, evanescente = e, flameación = f, grande = l, ninguno = n, colgante = p, revestimiento = s, zona = z

* Spore-print-color: negro = k, marrón = n, buff = b, chocolate = h, verde = r, naranja = o, púrpura = u, blanco = w, amarillo = y

* Población: abundante=a,agrupada=c,numerosa=n,dispersa=s,varias=v,solitaria=y

* Hábitat: pastos = g, hojas = L, prados = m, caminos = p, urbano = U, desperdicio = W, bosques = D

Como podemos ver, se trata de únicamente variables categóricas para todo el datase, por lo que no será necesario una normalización o estandarización de características numéricas (dado que no hay).

Verifiquemos en primera instancia si el dataset no tiene datos vacíos

In [4]:
 # Calcula la cantidad de valores nulos por columna
nulos_por_columna = data.isna().sum()

# Crea un DataFrame con los resultados
tabla_nulos = pd.DataFrame(nulos_por_columna, columns=['Valores Nulos'])

# Convierte la tabla de valores nulos en una tabla formateada
tabla_formateada = tabulate(tabla_nulos, headers='keys', tablefmt='fancy_grid')

# Imprime la tabla formateada
print(tabla_formateada)


╒══════════════════════════╤═════════════════╕
│                          │   Valores Nulos │
╞══════════════════════════╪═════════════════╡
│ class                    │               0 │
├──────────────────────────┼─────────────────┤
│ cap-shape                │               0 │
├──────────────────────────┼─────────────────┤
│ cap-surface              │               0 │
├──────────────────────────┼─────────────────┤
│ cap-color                │               0 │
├──────────────────────────┼─────────────────┤
│ bruises                  │               0 │
├──────────────────────────┼─────────────────┤
│ odor                     │               0 │
├──────────────────────────┼─────────────────┤
│ gill-attachment          │               0 │
├──────────────────────────┼─────────────────┤
│ gill-spacing             │               0 │
├──────────────────────────┼─────────────────┤
│ gill-size                │               0 │
├──────────────────────────┼─────────────────┤
│ gill-color 

Lo anterior nos indicaría que ninguna de las características presenta registros nulos, lo que indicaría que no es necesario dar  tratamiento de valores faltantes.

Seguido de esto, ahora observemos una tabla de frecuencias de los datos así como de las categorías y registros que hay.

In [5]:
# Calcula la descripción estadística y transpón el resultado
descripcion = data.describe().T

# Cambia los nombres de las columnas para una mejor presentación
descripcion.rename(columns={'count': 'Conteo', 'unique': 'Clases', 'top': 'Más frecuente', 'freq': 'Frequencia'}, inplace=True)

# Convierte la descripción en una tabla formateada
tabla = tabulate(descripcion, headers=descripcion.columns, tablefmt='fancy_grid', floatfmt=".0f")

# Imprime la tabla formateada
print(tabla)

╒══════════════════════════╤══════════╤══════════╤═════════════════╤══════════════╕
│                          │   Conteo │   Clases │ Más frecuente   │   Frequencia │
╞══════════════════════════╪══════════╪══════════╪═════════════════╪══════════════╡
│ class                    │     8124 │        2 │ e               │         4208 │
├──────────────────────────┼──────────┼──────────┼─────────────────┼──────────────┤
│ cap-shape                │     8124 │        6 │ x               │         3656 │
├──────────────────────────┼──────────┼──────────┼─────────────────┼──────────────┤
│ cap-surface              │     8124 │        4 │ y               │         3244 │
├──────────────────────────┼──────────┼──────────┼─────────────────┼──────────────┤
│ cap-color                │     8124 │       10 │ n               │         2284 │
├──────────────────────────┼──────────┼──────────┼─────────────────┼──────────────┤
│ bruises                  │     8124 │        2 │ f               │        

Una vez realizado el análisis anterior, lo siguiente que haremos es el prepocesamiento de los datos

En este caso, debemos codificar las variables categóricas en variables numéricas utilizando la técnica de codificación one-hot. También separaremos las características de las etiquetas

#### One Hot Encoding

In [18]:
# Obtener las variables categóricas
categorical_features = data.columns

# Aplicar one-hot encoding a todas las variables categóricas
df_encoded = pd.get_dummies(data, columns=categorical_features)

# Mostrar las dimensiones del conjunto de datos
print("Dimensiones del conjunto de datos antes de one-hot encoding:", data.shape)

# Mostrar las dimensiones del nuevo conjunto de datos
print("Dimensiones del conjunto de datos después de one-hot encoding:", df_encoded.shape)


print('El número de características despues del one-hot incrementó en',df_encoded.shape[1]-data.shape[1],)

Dimensiones del conjunto de datos antes de one-hot encoding: (8124, 23)
Dimensiones del conjunto de datos después de one-hot encoding: (8124, 119)
El número de características despues del one-hot incrementó en 96


In [19]:
# División en características y etiquetas
X = df_encoded.drop(['class_e', 'class_p'], axis=1)  # Características
y = df_encoded[['class_e', 'class_p']]  # Etiquetas codificadas

# División en conjuntos de entrenamiento, validación y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

# Muestra las dimensiones de los conjuntos
print("Dimensiones del conjunto de entrenamiento:", X_train.shape, y_train.shape)
print("Dimensiones del conjunto de validación:", X_val.shape, y_val.shape)
print("Dimensiones del conjunto de prueba:", X_test.shape, y_test.shape)

Dimensiones del conjunto de entrenamiento: (5686, 117) (5686, 2)
Dimensiones del conjunto de validación: (1219, 117) (1219, 2)
Dimensiones del conjunto de prueba: (1219, 117) (1219, 2)


Una vez que hemos generado los conjuntos de entrenamiento, validación y prueba, seguimos con la eliminación de características irrelevantes 

In [20]:
#### LO COMENTÉ PQ SIGUE NUEVO ANÄLISIS 

# # Extrae las características y las etiquetas
# X = data.drop('class', axis=1)  # Características
# y = data['class']               # Etiquetas

# # Codificación one-hot de las características
# encoder = OneHotEncoder()
# X_encoded = encoder.fit_transform(X)

# # Verifica las dimensiones del conjunto de características codificadas
# print(X_encoded.shape)


In [76]:
#### Solo si deseamos reducir la dimensionalidad 

#### PCA (Análisis de Componentes Principales) Solo aplica para variables continuas 

In [21]:
# Aplicar PCA
pca = PCA()
X_train_pca = pca.fit_transform(X_train)
X_val_pca = pca.transform(X_val)
X_test_pca = pca.transform(X_test)

# Obtener la varianza explicada por cada componente principal
explained_variance = pca.explained_variance_ratio_

# Determinar el número de componentes principales a mantener
n_components = 10  # Elige un número adecuado de componentes

# Reducir la dimensionalidad conservando solo los componentes principales seleccionados
pca = PCA(n_components=n_components)
X_train_pca = pca.fit_transform(X_train)
X_val_pca = pca.transform(X_val)
X_test_pca = pca.transform(X_test)

# Mostrar la varianza explicada acumulada
explained_variance_ratio_cumulative = np.cumsum(explained_variance)

# Imprimir la varianza explicada acumulada
print("Varianza explicada acumulada:", explained_variance_ratio_cumulative)

# Mostrar las dimensiones después de la reducción de dimensionalidad
print("Dimensiones después de la reducción de dimensionalidad:")
print("Conjunto de entrenamiento:", X_train_pca.shape)
print("Conjunto de validación:", X_val_pca.shape)
print("Conjunto de prueba:", X_test_pca.shape)

Varianza explicada acumulada: [0.16923355 0.30780259 0.40498079 0.46773915 0.50842163 0.54506831
 0.57663246 0.60426573 0.63067526 0.65152438 0.67198069 0.69052924
 0.70861288 0.72556368 0.74085354 0.75580548 0.77026285 0.78432715
 0.79761704 0.80931914 0.82029341 0.83102787 0.84141787 0.85090315
 0.8596872  0.86822895 0.87626015 0.88378811 0.89117923 0.89814722
 0.90475332 0.91094487 0.91700549 0.92255819 0.92783366 0.93305515
 0.93770945 0.94227526 0.94655355 0.95067535 0.95473419 0.95859314
 0.96209863 0.96552991 0.96884322 0.97190614 0.97462495 0.97692121
 0.97906268 0.98110586 0.98288777 0.98454762 0.98604706 0.9873442
 0.98857715 0.98969781 0.99075216 0.99165035 0.99244829 0.99316975
 0.99382843 0.99446423 0.99507151 0.99565156 0.99619374 0.99667212
 0.99711529 0.99750349 0.99784386 0.99816788 0.99845386 0.99869769
 0.99890306 0.99909278 0.9992541  0.99940974 0.99954429 0.99966172
 0.99975856 0.99982782 0.99988841 0.99993565 0.99997126 0.99999684
 1.         1.         1.        

#### Test chi cuadrado

In [22]:


# Aplicar test chi-cuadrado para seleccionar las K mejores características
k = 40  # Número de características a seleccionar
selector = SelectKBest(chi2, k=k)
X_train_selected = selector.fit_transform(X_train, y_train)
X_val_selected = selector.transform(X_val)
X_test_selected = selector.transform(X_test)

# Obtener los índices de las características seleccionadas
selected_feature_indices = selector.get_support(indices=True)

# Obtener los nombres de las características seleccionadas
selected_feature_names = X_train.columns[selected_feature_indices]

# Mostrar las características seleccionadas
print("Características seleccionadas:")
print(selected_feature_names)

# Mostrar las dimensiones después de la selección de características
print("Dimensiones después de la selección de características:")
print("Conjunto de entrenamiento:", X_train_selected.shape)
print("Conjunto de validación:", X_val_selected.shape)
print("Conjunto de prueba:", X_test_selected.shape)


Características seleccionadas:
Index(['bruises_f', 'bruises_t', 'odor_a', 'odor_f', 'odor_l', 'odor_n',
       'odor_s', 'odor_y', 'gill-spacing_w', 'gill-size_b', 'gill-size_n',
       'gill-color_b', 'gill-color_n', 'gill-color_u', 'gill-color_w',
       'stalk-root_?', 'stalk-root_c', 'stalk-root_e',
       'stalk-surface-above-ring_k', 'stalk-surface-above-ring_s',
       'stalk-surface-below-ring_k', 'stalk-surface-below-ring_s',
       'stalk-color-above-ring_b', 'stalk-color-above-ring_g',
       'stalk-color-above-ring_n', 'stalk-color-above-ring_p',
       'stalk-color-below-ring_b', 'stalk-color-below-ring_g',
       'stalk-color-below-ring_p', 'ring-number_t', 'ring-type_l',
       'ring-type_p', 'spore-print-color_h', 'spore-print-color_k',
       'spore-print-color_n', 'spore-print-color_w', 'population_a',
       'population_n', 'population_v', 'habitat_p'],
      dtype='object')
Dimensiones después de la selección de características:
Conjunto de entrenamiento: (5686, 40)

In [23]:
# Obtener solo las características seleccionadas
X_train_selected = X_train[selected_feature_names]  # Reemplaza con las características seleccionadas
X_val_selected = X_val[selected_feature_names]      # Reemplaza con las características seleccionadas
X_test_selected = X_test[selected_feature_names]    # Reemplaza con las características seleccionadas

# Verificar las dimensiones
print("Dimensiones después de la selección de características:")
print("Conjunto de entrenamiento:", X_train_selected.shape)
print("Conjunto de validación:", X_val_selected.shape)
print("Conjunto de prueba:", X_test_selected.shape)

Dimensiones después de la selección de características:
Conjunto de entrenamiento: (5686, 40)
Conjunto de validación: (1219, 40)
Conjunto de prueba: (1219, 40)


#### Diseño de la red neuronal completamente conectada 

In [24]:
from keras.models import Sequential
from keras.layers import Dense

# Crear el modelo
model = Sequential()

# Agregar la capa oculta
model.add(Dense(64, activation='relu', input_shape=(40,)))

# Agregar la capa de salida
model.add(Dense(2, activation='softmax'))

# Compilar el modelo
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Mostrar un resumen del modelo
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                2624      
                                                                 
 dense_1 (Dense)             (None, 2)                 130       
                                                                 
Total params: 2,754
Trainable params: 2,754
Non-trainable params: 0
_________________________________________________________________


In [32]:
# Entrenar el modelo
history = model.fit(X_train_selected, y_train, batch_size=12, epochs=100, validation_data=(X_val_selected, y_val))


# Obtener las métricas de precisión y pérdida
train_accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
train_loss = history.history['loss']
val_loss = history.history['val_loss']

# Imprimir las métricas
print("Precisión de entrenamiento:", train_accuracy[-1])
print("Precisión de validación:", val_accuracy[-1])
print("Pérdida de entrenamiento:", train_loss[-1])
print("Pérdida de validación:", val_loss[-1])


Precisión de entrenamiento: 1.0
Precisión de validación: 1.0
Pérdida de entrenamiento: 0.0
Pérdida de validación: 0.0


In [34]:
# Evaluar el modelo en el conjunto de prueba
test_loss, test_accuracy = model.evaluate(X_test_selected, y_test)
print("Precisión en el conjunto de prueba:", test_accuracy)
print("Pérdida en el conjunto de prueba:", test_loss)


AttributeError: 'KerasClassifier' object has no attribute 'evaluate'

In [35]:
from sklearn.model_selection import cross_val_score
from keras.wrappers.scikit_learn import KerasClassifier

# Definir la función para crear el modelo
def create_model():
       # Crear el modelo
    model = Sequential()

    # Agregar la primera capa oculta
    model.add(Dense(64, activation='relu', input_shape=(40,)))

    # Agregar la segunda capa oculta
    model.add(Dense(32, activation='relu'))

    # Agregar la capa de salida
    model.add(Dense(2, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Crear el clasificador de Keras
model = KerasClassifier(build_fn=create_model, epochs=10, batch_size=64, verbose=0)

# Realizar la validación cruzada
scores = cross_val_score(model, X_train_selected, y_train, cv=5)

# Imprimir los resultados
print("Precisión promedio:", scores.mean())
print("Desviación estándar de la precisión:", scores.std())


C:\Users\adame\AppData\Local\Temp\ipykernel_9556\2321397732.py:21: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=create_model, epochs=10, batch_size=64, verbose=0)


Precisión promedio: 1.0
Desviación estándar de la precisión: 0.0


In [31]:


# Crear el modelo
model = Sequential()

# Agregar la primera capa oculta
model.add(Dense(64, activation='relu', input_shape=(40,)))

# Agregar la segunda capa oculta
model.add(Dense(32, activation='relu'))

# Agregar la capa de salida
model.add(Dense(2, activation='softmax'))

# Compilar el modelo
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Imprimir un resumen del modelo
model.summary()


Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_21 (Dense)            (None, 64)                2624      
                                                                 
 dense_22 (Dense)            (None, 32)                2080      
                                                                 
 dense_23 (Dense)            (None, 2)                 66        
                                                                 
Total params: 4,770
Trainable params: 4,770
Non-trainable params: 0
_________________________________________________________________


In [32]:
# Entrenar el modelo
history = model.fit(X_train_selected, y_train, batch_size=64, epochs=10, validation_data=(X_val_selected, y_val))


# Obtener las métricas de precisión y pérdida
train_accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
train_loss = history.history['loss']
val_loss = history.history['val_loss']

# Imprimir las métricas
print("Precisión de entrenamiento:", train_accuracy[-1])
print("Precisión de validación:", val_accuracy[-1])
print("Pérdida de entrenamiento:", train_loss[-1])
print("Pérdida de validación:", val_loss[-1])


Epoch 1/10
89/89 [==============================] - 2s 8ms/step - loss: 0.2531 - accuracy: 0.9071 - val_loss: 0.0762 - val_accuracy: 0.9811
Epoch 2/10
89/89 [==============================] - 0s 4ms/step - loss: 0.0452 - accuracy: 0.9866 - val_loss: 0.0271 - val_accuracy: 0.9992
Epoch 3/10
89/89 [==============================] - 0s 4ms/step - loss: 0.0180 - accuracy: 0.9974 - val_loss: 0.0108 - val_accuracy: 0.9992
Epoch 4/10
89/89 [==============================] - 0s 3ms/step - loss: 0.0084 - accuracy: 0.9984 - val_loss: 0.0058 - val_accuracy: 0.9992
Epoch 5/10
89/89 [==============================] - 0s 4ms/step - loss: 0.0044 - accuracy: 0.9993 - val_loss: 0.0033 - val_accuracy: 1.0000
Epoch 6/10
89/89 [==============================] - 0s 3ms/step - loss: 0.0025 - accuracy: 1.0000 - val_loss: 0.0021 - val_accuracy: 1.0000
Epoch 7/10
89/89 [==============================] - 0s 3ms/step - loss: 0.0016 - accuracy: 1.0000 - val_loss: 0.0014 - val_accuracy: 1.0000
Epoch 8/10
89/89 [==

In [33]:
# Evaluar el modelo en el conjunto de prueba
test_loss, test_accuracy = model.evaluate(X_test_selected, y_test)
print("Precisión en el conjunto de prueba:", test_accuracy)
print("Pérdida en el conjunto de prueba:", test_loss)


39/39 [==============================] - 0s 2ms/step - loss: 5.4744e-04 - accuracy: 1.0000
Precisión en el conjunto de prueba: 1.0
Pérdida en el conjunto de prueba: 0.000547439674846828


In [55]:
from sklearn.model_selection import cross_val_score
from keras.wrappers.scikit_learn import KerasClassifier

# Definir la función para crear el modelo
def create_model():
       # Crear el modelo
    model = Sequential()

    # Agregar la primera capa oculta
    model.add(Dense(64, activation='relu', input_shape=(40,)))

    # Agregar la segunda capa oculta
    model.add(Dense(32, activation='relu'))

    # Agregar la capa de salida
    model.add(Dense(2, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam',  metrics=['accuracy'])
    return model

# Crear el clasificador de Keras
model = KerasClassifier(build_fn=create_model, epochs=10, batch_size=64, verbose=0)

# Realizar la validación cruzada
scores = cross_val_score(model, X_train_selected, y_train, cv=5)

# Imprimir los resultados
print("Precisión promedio:", scores.mean())
print("Desviación estándar de la precisión:", scores.std())


C:\Users\adame\AppData\Local\Temp\ipykernel_9556\2321397732.py:21: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=create_model, epochs=10, batch_size=64, verbose=0)


Precisión promedio: 1.0
Desviación estándar de la precisión: 0.0
